### This file querries all 300K-ish CVE entires in the NVD and store the top 3 matches into mongo.

In [ ]:
### Include all necessary imports ###

import sys
from pathlib import Path
from pymongo import MongoClient


### Get the correct directory resolved for imports ###
project_root = Path().resolve()
while not (project_root / 'embedding_pipeline').exists() and project_root != project_root.parent:
    project_root = project_root.parent

sys.path.append(str(project_root))

from embedding_pipeline.weaviate_db.weaviate_config import connect_to_local_weaviate_client
from embedding_pipeline.weaviate_db.weaviate_config import verify_weaviate_client_ready
from embedding_pipeline.weaviate_db.weaviate_config import close_weaviate_client
from embedding_pipeline.weaviate_db.weaviate_config import list_weaviate_collections, inspect_collection_properties, retrieve_existing_weaviate_collection





In [ ]:
### Connect to local weavite client ###
local_client = connect_to_local_weaviate_client()
list_weaviate_collections(local_client)

### Get the local weavite collection name ###
WEAVIATE_COLL_NAME = "FOSS_vectors"


### Connect to MongoDB clinet ###

uri = "mongodb://localhost:27017"
mongodb_client = MongoClient(uri)


try:
    mongodb_client.admin.command("ping")
    print("Connected to MongoDB!")
except :
    print("❌ Could not connect to MongoDB:")

### Get the CVE database ###
NVD_CVE_DB = mongodb_client["nvdcve"]

### Get / create the collection ###
enriched_cve_collection = NVD_CVE_DB["enriched_cve_entries"]

